# YOLOv10 Training and Inference Notebook

This notebook contains steps to prepare, train, evaluate, and export a YOLOv10 object detection model for pothole detection. Sections are organized to make it easy to run step-by-step on Google Colab or a similar environment. Do not modify code cells unless you know what you are doing.


## Environment and Drive Mount

Mount your Google Drive to access datasets, models, and outputs. If you are running locally, skip the drive mount and update paths accordingly.


In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!python --version

In [ ]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.6 LTS
Release:	20.04
Codename:	focal


In [ ]:
!ls

drive/  sample_data/


In [ ]:
!cd /content/drive/MyDrive/PDS/

/content/drive/MyDrive/PDS


In [ ]:
!ls

 Datasets/   data.yaml  'Inference Images'/   Models/   settings.yaml   train/   Yolov10.ipynb


In [ ]:
# Check the configuration of data.yaml file
!cat /content/drive/MyDrive/PDS/data.yaml

## System and GPU Checks

Check the runtime environment and GPU availability. These steps help confirm that CUDA and a compatible GPU are present before training.


In [ ]:
!pip install nvidia-smi

In [ ]:
!nvidia-smi

In [ ]:
import subprocess

# Run nvidia-smi command to check GPU status
result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if result.returncode == 0:
    print("GPU is detected:")
    print(result.stdout.decode("utf-8"))
else:
    print("GPU is not detected.")
    print(result.stderr.decode("utf-8"))

GPU is detected:
Fri May 24 14:18:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   35C    P8              16W /  72W |      0MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f"GPU Name: {torch.cuda.get_device_name(device)}")
    print(
        f"GPU Memory Total: {torch.cuda.get_device_properties(device).total_memory / (1024 ** 2):.2f} MB"
    )
    print(
        f"GPU Memory Total: {torch.cuda.get_device_properties(device).total_memory / (1024 ** 3):.2f} GB"
    )
else:
    print("No GPU available.")


GPU Name: NVIDIA L4
GPU Memory Total: 22699.875 MB


## Install Dependencies

Install required Python packages such as `ultralytics` and `roboflow`. Run these before executing dataset download, training, or inference cells.


In [ ]:
!pip install ultralytics
!pip install roboflow

In [ ]:
!pip install --upgrade ultralytics

## Dataset Download and Preparation

Download the dataset from Roboflow or provide your own dataset path. Ensure the `data.yaml` file paths are correct before training.


In [ ]:
# Download and Load Dataset (8315 Images for testing model training, final training with 50k images)
# url: https://universe.roboflow.com/pothole-annotation-np4pr/pds-new/dataset/1

from roboflow import Roboflow

try:
    rf = Roboflow(api_key="...")  # Enter your API key here
    project = rf.workspace("...").project(
        "..."
    )  # Enter your workspace and project name here

    version = project.version(1)
    dataset = version.download("yolov10")
except Exception as e:
    print("Error downloading dataset:", e)

# Uses old ultralytics version so manually upload dataset

# If you have a dataset on device,
# comment out the above lines
# and upload your dataset manually

## Training the YOLOv10 Model

Train the model using the chosen YOLOv10 architecture. Adjust `model`, `data`, `epochs`, and `imgsz` as needed.


In [ ]:
# Train the model using YOLOv10m architecture as base model and the custom dataset
# Change the model path if you want to use a different architecture (yolov10n, yolov10s, yolov10b, yolov10l, yolov10x)
# Change the data path if your data.yaml file is in a different location

!yolo task=detect \
     mode=train \
     model=/content/drive/MyDrive/PDS/Models/yolov10m.pt \
     data=/content/drive/MyDrive/PDS/data.yaml \
     epochs=30 \
     imgsz=640


Ultralytics YOLOv8.2.38 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/PDS/Models/yolov10m.pt, data=/content/drive/MyDrive/PDS/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

## Testing and Inference

Load the trained model and run inference on example images. Use `result.show()` to visualize predictions and `result.save()` to store outputs.


In [ ]:
# Testing Results of Model

from ultralytics import YOLO

# Load a model
model = YOLO(
    "/content/drive/MyDrive/PDS/best.pt"
)  # Newly trained & finetuned YOLOv10m model


# Run batched inference on a list of images
results = model(
    ["/content/drive/MyDrive/PDS/Inference Images/p1.jpg"]
)  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs

    result.show()  # display to screen

    result.save(
        filename="/content/drive/MyDrive/PDS/Inference Images/p1-result.jpeg"
    )  # save to disk

## Upload Trained Weights

Optionally upload trained weights to Roboflow or another model hosting service for deployment.


In [ ]:
# Upload weights (Model to roboflow)

from roboflow import Roboflow

rf = Roboflow(api_key="...")  # Enter your API key here
project = rf.workspace("...").project(
    "..."
)  # Enter your workspace and project name here
project.version(1).deploy(
    model_type="yolov10m", model_path="/content/drive/MyDrive/PDS/train/"
)

## Export Model to EdgeTPU and ONNX

Export the trained `best.pt` to formats suitable for edge deployment, such as EdgeTPU (tflite) and ONNX.


In [ ]:
# Export Model to EdgeTPU & ONNX

from ultralytics import YOLO

model = YOLO(
    "/content/drive/MyDrive/PDS/best.pt"
)  # Newly trained & finetuned YOLOv10m model

model.export(
    format="edgetpu"
)  # Export to EdgeTPU (tflite model for Google Coral Accelerator)

model.export(format="onnx")  # Export to ONNX format

## Archive Trained Models

Zip the training output folder to download or share the trained model artifacts easily.


In [ ]:
# Zip the trained model folder for easy download
!zip -r train.zip /content/runs/detect/train